# Historical Images of 65th St and Woodlawn Ave Community Garden

In this notebook, we are going to use Google Earth Engine and the Python package `geemap` to extract images of the garden and create a timelapse `.gif`

We will use USDA's National Agriculture Imagery Program (NAIP) dataset

### Import Google Earth Engine and `geemap`

In [3]:
import ee
# Trigger the authentication flow.
ee.Authenticate()
# Initialize the library.
ee.Initialize()



Successfully saved authorization token.


In [4]:
import geemap

### Import Coordinates

In [5]:
import os
from os.path import dirname, join as pjoin
import json # we are using JSON instead of GeoJSON in this example

JSON file `coords.json` is formatted like so:
```
{
  "<location_name>": {
    "lon": <longitude in decimal degrees>,
    "lat": <latitude in decimal degress>,
    "scale": <scale in meters>
  },
    "<another_location_name>": {
    "lon": <longitude in decimal degrees>,
    "lat": <latitude in decimal degress>,
    "scale": <scale in meters>
  }
  ...
}
```

In [6]:
coords_path = "coords.json"
with open(coords_path, 'r') as coords_file:
    coords = json.load(coords_file)

output_folder = "output"

## Create NAIP Timelapse

### View Location with buffer (indicated by `scale` parameter)
validate location and region of interest

In [7]:
Map = geemap.Map()
Map
location_name = 'theodore_stone_forest'#'sundown_meadow'#'community_garden'
lon = coords[location_name]["lon"]
lat = coords[location_name]["lat"]
scale = coords[location_name]["scale"]
geom_point = ee.Geometry.Point(lon,lat)
roi = geom_point.buffer(scale)
Map.addLayer(roi)
Map.centerObject(roi)
Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

### Create gif of timelapse

In [8]:
im_col = geemap.naip_timeseries(roi,start_year=2005,RGBN=True)
images = im_col.toList(im_col.size())
dates = geemap.image_dates(im_col).getInfo()
size = images.size().getInfo()
for i in range(size):
    image = ee.Image(images.get(i))
    Map.addLayer(image, {'bands': ['R','G','B']}, dates[i][:4],shown=False)

timelapse = geemap.naip_timelapse(
    roi, start_year=2005, out_gif=pjoin(output_folder,f"{location_name}-{scale}m-naip.gif"), bands=['R','G','B'], frames_per_second=1,fading=1,)
geemap.show_image(timelapse)

Generating URL...
Please wait ...
The GIF image has been saved to: c:\Users\marin\Documents\_MJMS\Dev\65th-woodlawn\get-satellite-images\output\theodore_stone_forest-600m-naip.gif
ffmpeg is not installed on your computer.
[WinError 32] The process cannot access the file because it is being used by another process: 'C:\\Users\\marin\\AppData\\Local\\Temp\\theodore_stone_forest-600m-naip'


Output()

### NDVI on NAIP
Normalized Difference Vegetation Index

In [12]:
# Create a map centered at (lat, lon).
Map = geemap.Map(center=[lat,lon], zoom=4)
location_name = 'community_garden'
lon = coords[location_name]["lon"]
lat = coords[location_name]["lat"]
first = (ee.ImageCollection('USDA/NAIP/DOQQ')
         .filterBounds(ee.Geometry.Point(lon,lat))
         .filterDate('2008-01-01', '2022-12-31')
         .sort('CLOUDY_PIXEL_PERCENTAGE')
         .first())


# Define the visualization parameters.
vizParams =  {
  min: 0.0,
  max: 255.0,
}

nir = first.select('N')
red = first.select('R')
ndvi = nir.subtract(red).divide(nir.add(red))
vizParams =  {
  'min': -1,
  'max': 1,
  'palette':['blue','white','green']
}
# Center the map and display the image.
Map.setCenter(lon,lat,19) 
# Map.addLayer(first, vizParams, 'color')
Map.addLayer(ndvi, vizParams, 'ndvi')

# Display the map.
Map

Map(center=[41.777406694809436, -87.59572748186986], controls=(WidgetControl(options=['position', 'transparent…